# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [61]:
# imports
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import gradio as gr

In [2]:

# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')
base_url = os.getenv('OPENROUTER_BASE_URL')

if api_key and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [3]:
# set up list of models
MODEL_GPT = 'gpt-4o-mini'
MODEL_CLAUDE_SONNET = 'claude-sonnet-4.5'

In [4]:
# create openRouter object
openRouter = OpenAI(base_url=base_url, api_key=api_key)

In [ ]:
# define a get_ticket_price function to be a tool for the LLM
def get_ticket_price(destination_city):
    ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}
    price = ticket_prices.get(destination_city.lower(), "Unknown ticket price")
    return f"The price of a ticket to {destination_city} is {price}"

# here's a particular dictionary structure that's required to describe our function:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# define a handle_tool_calls function to be used to handle LLM tool call
def handle_tool_calls(message):
    responses = []
    for tool in message.tool_calls:
        if tool.function.name == "get_ticket_price":
            arguments = json.loads(tool.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool.id
            })
    return responses


# And this is included in a list of tools:
tools = [{"type": "function", "function": price_function}]

In [ ]:
def stream_question_and_answer(prompt, model):
    messages = [
        {"role": "system", "content": "You are a technical assistant"},
        {"role": "user", "content": prompt}
    ]

    selectedModel = None
    match model:
        case "GPT":
            selectedModel = MODEL_GPT
        case "Claude":
            selectedModel = MODEL_CLAUDE_SONNET
        case _:
            raise ValueError("Unknown model")

    out = ""
    while True:
        response = openRouter.chat.completions.create(
            model=selectedModel,
            messages=messages,
            stream=True,
            tools=tools
        )

        delta_tool_calls = None
        for chunk in response:                  
            if chunk.choices[0].delta.content:
                out += chunk.choices[0].delta.content or ''
                yield out

            if chunk.choices[0].delta.tool_calls:
                for tool_call in chunk.choices[0].delta.tool_calls:
                    
                    if tool_call.id is not None:
                        delta_tool_calls = chunk.choices[0].delta
                        
                    # # for streaming response, arguments stream as partial JSON
                    if tool_call.function.arguments:
                        delta_tool_calls.tool_calls[0].function.arguments = delta_tool_calls.tool_calls[0].function.arguments + tool_call.function.arguments

        if delta_tool_calls:
            responses = handle_tool_calls(delta_tool_calls)
            messages.append(delta_tool_calls)
            messages.extend(responses)
            continue
        break
        

In [130]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_question_and_answer,
    title="LLMs", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
        ["Explain the Transformer architecture to a layperson", "GPT"],
        ["Explain the Transformer architecture to an aspiring AI engineer", "Claude"]
    ], 
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
